In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [2]:
spark = SparkSession.builder.appName("elt").getOrCreate()

In [48]:
input_data1 = "data/log-data/*.json"

In [181]:
df=spark.read.json(input_data1, multiLine=True)
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [182]:
df.select("page").distinct().toPandas()

,page
0,Home
1,Login
2,NextSong


In [219]:
from pyspark.sql.functions import *
dff = df.filter((col("page") =='NextSong') & col("userId").isNotNull()).groupby('userId').agg({'ts':'max'}) 

In [221]:
dff.printSchema()

root
 |-- userId: string (nullable = true)
 |-- max(ts): long (nullable = true)



In [227]:
dfl=df.join(dff, (df.ts==col("max(ts)")) & (df.userId==dff.userId), 'right').select(df.userId.alias("user_id"), col("firstName").alias("first_name"), 
                                             col("lastName").alias("last_name"), "gender", "level")

In [ ]:
## Break the problem in parts
##Part-1 I need incremental songplay_id
##

In [243]:
## Getting normal columns from table 
dft=df.filter("page='NextSong'")
dft.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [286]:
## Solving Timestamp problem
# get_timestamp = udf()
date_time = df.select(from_unixtime(col("ts")/1000).alias('tsm'))

In [275]:
new_df = dft.join(dfs,(dft.artist==dfs.artist_name) & (dft.song==dfs.title), 'left')\
    .select(from_unixtime(col("ts")/1000).alias("start_time"), col("userId").alias("user_id"), "level", dfs.song_id, dfs.artist_id,\
     col("sessionId").alias("session_id"), "location", col("userAgent").alias("user_agent"))

In [298]:
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
#start_time, hour, day, week, month, year, weekday
time_table = date_time.select(col("tsm").alias("start_time"), hour(col("tsm")).alias("hour"), dayofmonth(col("tsm")).alias("day"),\
                             weekofyear(col("tsm")).alias("week"), month(col("tsm")).alias("month"), year(col("tsm")).alias("year"),\
                             dayofweek(col("tsm")).alias("weekday"))

In [299]:
time_table.show()

+-------------------+----+---+----+-----+----+-------+
|         start_time|hour|day|week|month|year|weekday|
+-------------------+----+---+----+-----+----+-------+
|2018-11-15 11:30:26|  11| 15|  46|   11|2018|      5|
|2018-11-21 11:27:34|  11| 21|  47|   11|2018|      4|
|2018-11-14 11:03:22|  11| 14|  46|   11|2018|      4|
|2018-11-28 11:00:15|  11| 28|  48|   11|2018|      4|
|2018-11-05 11:33:12|  11|  5|  45|   11|2018|      2|
|2018-11-13 11:36:57|  11| 13|  46|   11|2018|      3|
|2018-11-30 11:22:07|  11| 30|  48|   11|2018|      6|
|2018-11-16 11:00:57|  11| 16|  46|   11|2018|      6|
|2018-11-20 11:00:42|  11| 20|  47|   11|2018|      3|
|2018-11-24 11:45:00|  11| 24|  47|   11|2018|      7|
|2018-11-29 11:00:57|  11| 29|  48|   11|2018|      5|
|2018-11-19 12:54:28|  12| 19|  47|   11|2018|      2|
|2018-11-27 11:52:12|  11| 27|  48|   11|2018|      3|
|2018-11-23 11:07:25|  11| 23|  47|   11|2018|      6|
|2018-11-09 11:06:17|  11|  9|  45|   11|2018|      6|
|2018-11-2

In [306]:
## Artist Table
#artist_id, name, location, lattitude, longitude
artist = dfs.filter("artist_id is NOT NULL").select("artist_id", col("artist_location").alias("location"), col("artist_latitude").alias("latitude"),\
           col("artist_longitude").alias("longitude"))


In [334]:
a1 = dfs.alias("one").filter("artist_id is NOT NULL").groupby("artist_id").agg({'year':'max'})
a1.show()


+------------------+---------+
|         artist_id|max(year)|
+------------------+---------+
|AR9AWNF1187B9AB0B4|        0|
+------------------+---------+



In [339]:
## logic to get artist' latest details by year, 
## when artist has two updates per year, records are taken as sort order whichever appear first

a2 = dfs.alias("one").filter("artist_id is NOT NULL").groupby("artist_id").agg({'year':'max'})\
.join(dfs.alias("two"), (col("one.artist_id")==col("two.artist_id")) & (col("max(year)")==col("two.year")), 'right')\
.select("one.artist_id", col("artist_location").alias("location"), col("artist_latitude").alias("latitude"),\
           col("artist_longitude").alias("longitude"))

In [ ]:
## let us optimize query

In [333]:
## Song Table
dfs.count()

71